In [1]:
from step_1 import load_and_prepare_data
from step_2 import feature_selection_rfecv
from step_3 import train_and_select_model
from step_4 import tune_and_blend_models
from step_5 import evaluate_model

In [2]:
from sklearn.metrics import f1_score as metric_func

target = 'RainTomorrow'

df = load_and_prepare_data(file_path = 'weatherAUS.csv', target_column=target, date_column = 'Date', fill_median= True, drop_empty=True, encode_objects = True)



df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,RainToday_No,RainToday_Yes
0,13.4,22.9,0.6,4.8,8.4,44.0,20.0,24.0,71.0,22.0,...,False,False,False,False,False,False,True,False,True,False
1,7.4,25.1,0.0,4.8,8.4,44.0,4.0,22.0,44.0,25.0,...,False,False,False,False,False,False,False,True,True,False
2,12.9,25.7,0.0,4.8,8.4,46.0,19.0,26.0,38.0,30.0,...,False,False,False,False,False,False,False,True,True,False
3,9.2,28.0,0.0,4.8,8.4,24.0,11.0,9.0,45.0,16.0,...,False,False,False,False,False,False,False,False,True,False
4,17.5,32.3,1.0,4.8,8.4,41.0,7.0,20.0,82.0,33.0,...,False,False,False,False,False,False,False,False,True,False


In [3]:
selected_features = feature_selection_rfecv(data = df, target_column=target, metric = metric_func, min_features=10, selector_step=10)
selected_features

Index(['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'year', 'month', 'day', 'RainToday_No'],
      dtype='object')

In [4]:
df = df[selected_features.to_list()+[target]]

In [3]:
df = df[['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'year', 'month', 'day', 'RainToday_No'] + [target]]

In [6]:
cutoff = int(0.2 * len(df))
df_val = df.iloc[:cutoff]
df_cut = df.iloc[cutoff:]

In [7]:
model = train_and_select_model(data = df_cut, target_column=target, metric=metric_func)

[LightGBM] [Info] Number of positive: 17553, number of negative: 61621
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004499 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2572
[LightGBM] [Info] Number of data points in the train set: 79174, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.221702 -> initscore=-1.255778
[LightGBM] [Info] Start training from score -1.255778
Лучшая модель: CatBoost с значением метрики 0.6406543309198176


In [7]:
blender = tune_and_blend_models(data = df_cut, target_column=target, metric=metric_func, n_trials=3)

[I 2024-12-29 16:38:42,175] A new study created in memory with name: no-name-5a1eedc7-1afa-4891-9d3c-ead00860920e


0:	learn: 0.6222422	total: 10.3ms	remaining: 4.04s
1:	learn: 0.5674669	total: 18.7ms	remaining: 3.65s
2:	learn: 0.5249381	total: 27.3ms	remaining: 3.55s
3:	learn: 0.4918580	total: 37.4ms	remaining: 3.64s
4:	learn: 0.4643897	total: 46.6ms	remaining: 3.61s
5:	learn: 0.4432626	total: 55.5ms	remaining: 3.58s
6:	learn: 0.4263628	total: 64.5ms	remaining: 3.56s
7:	learn: 0.4130030	total: 74.1ms	remaining: 3.56s
8:	learn: 0.4028946	total: 83ms	remaining: 3.54s
9:	learn: 0.3951009	total: 92.4ms	remaining: 3.54s
10:	learn: 0.3873885	total: 102ms	remaining: 3.54s
11:	learn: 0.3814897	total: 111ms	remaining: 3.53s
12:	learn: 0.3762466	total: 120ms	remaining: 3.5s
13:	learn: 0.3718411	total: 130ms	remaining: 3.51s
14:	learn: 0.3685219	total: 139ms	remaining: 3.51s
15:	learn: 0.3649666	total: 148ms	remaining: 3.48s
16:	learn: 0.3621626	total: 157ms	remaining: 3.47s
17:	learn: 0.3599357	total: 166ms	remaining: 3.46s
18:	learn: 0.3579506	total: 175ms	remaining: 3.45s
19:	learn: 0.3561767	total: 184ms	

[I 2024-12-29 16:38:45,845] Trial 0 finished with value: 0.625879805089334 and parameters: {'depth': 6, 'learning_rate': 0.0951207163345817, 'iterations': 393}. Best is trial 0 with value: 0.625879805089334.


0:	learn: 0.6844825	total: 8.92ms	remaining: 1.44s
1:	learn: 0.6762850	total: 15.8ms	remaining: 1.27s
2:	learn: 0.6680272	total: 24.2ms	remaining: 1.28s
3:	learn: 0.6600407	total: 32.6ms	remaining: 1.29s
4:	learn: 0.6524912	total: 40.7ms	remaining: 1.28s
5:	learn: 0.6449178	total: 48.5ms	remaining: 1.26s
6:	learn: 0.6378115	total: 56.7ms	remaining: 1.25s
7:	learn: 0.6308734	total: 65.1ms	remaining: 1.25s
8:	learn: 0.6240419	total: 73.4ms	remaining: 1.25s
9:	learn: 0.6174533	total: 81.6ms	remaining: 1.24s
10:	learn: 0.6110435	total: 89.5ms	remaining: 1.23s
11:	learn: 0.6048228	total: 97.8ms	remaining: 1.22s
12:	learn: 0.5987459	total: 106ms	remaining: 1.21s
13:	learn: 0.5928910	total: 114ms	remaining: 1.2s
14:	learn: 0.5871884	total: 122ms	remaining: 1.2s
15:	learn: 0.5818000	total: 130ms	remaining: 1.18s
16:	learn: 0.5763213	total: 139ms	remaining: 1.18s
17:	learn: 0.5710813	total: 147ms	remaining: 1.17s
18:	learn: 0.5660172	total: 154ms	remaining: 1.16s
19:	learn: 0.5611360	total: 163

[I 2024-12-29 16:38:47,287] Trial 1 finished with value: 0.5815602836879432 and parameters: {'depth': 8, 'learning_rate': 0.016445845403801215, 'iterations': 162}. Best is trial 0 with value: 0.625879805089334.


...

Лучшие параметры для LightGBM: {'num_leaves': 90, 'learning_rate': 0.07245370197423232, 'n_estimators': 358}
[LightGBM] [Info] Number of positive: 17553, number of negative: 61621
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004445 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2572
[LightGBM] [Info] Number of data points in the train set: 79174, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.221702 -> initscore=-1.255778
[LightGBM] [Info] Start training from score -1.255778


[I 2024-12-29 16:41:57,304] A new study created in memory with name: no-name-3e673ddd-c71e-49ed-943e-564751b53ff9
[I 2024-12-29 16:41:58,212] Trial 0 finished with value: 0.638575036554566 and parameters: {'max_depth': 6, 'learning_rate': 0.0951207163345817, 'n_estimators': 393}. Best is trial 0 with value: 0.638575036554566.
[I 2024-12-29 16:41:59,094] Trial 1 finished with value: 0.5955768418087054 and parameters: {'max_depth': 8, 'learning_rate': 0.016445845403801215, 'n_estimators': 162}. Best is trial 0 with value: 0.638575036554566.
[I 2024-12-29 16:41:59,721] Trial 2 finished with value: 0.6224021592442645 and parameters: {'max_depth': 4, 'learning_rate': 0.08675143843171859, 'n_estimators': 341}. Best is trial 0 with value: 0.638575036554566.
[I 2024-12-29 16:42:02,383] Trial 3 finished with value: 0.5514328808446456 and parameters: {'max_depth': 8, 'learning_rate': 0.0030378649352844423, 'n_estimators': 488}. Best is trial 0 with value: 0.638575036554566.
[I 2024-12-29 16:42:0

Лучшие параметры для XGBoost: {'max_depth': 10, 'learning_rate': 0.0985707332215553, 'n_estimators': 487}


[I 2024-12-29 16:42:33,209] A new study created in memory with name: no-name-80c07a21-8dac-43c8-bdba-3ef25d4df7d2
[I 2024-12-29 16:42:33,437] Trial 0 finished with value: 0.6564238410596026 and parameters: {'coeff_0': 0.3745401188473625, 'coeff_1': 0.9507143064099162, 'coeff_2': 0.7319939418114051}. Best is trial 0 with value: 0.6564238410596026.
[I 2024-12-29 16:42:33,658] Trial 1 finished with value: 0.6338631651693158 and parameters: {'coeff_0': 0.5986584841970366, 'coeff_1': 0.15601864044243652, 'coeff_2': 0.15599452033620265}. Best is trial 0 with value: 0.6564238410596026.
[I 2024-12-29 16:42:33,905] Trial 2 finished with value: 0.6541313559322034 and parameters: {'coeff_0': 0.05808361216819946, 'coeff_1': 0.8661761457749352, 'coeff_2': 0.6011150117432088}. Best is trial 0 with value: 0.6564238410596026.
[I 2024-12-29 16:42:34,129] Trial 3 finished with value: 0.6546895640686922 and parameters: {'coeff_0': 0.7080725777960455, 'coeff_1': 0.020584494295802447, 'coeff_2': 0.96990985

Значение метрики у блендинга на тестовой выборке: 0.6566887417218543


In [8]:
res1 = evaluate_model(model, df_val.drop(columns=[target]), df_val[target], metric_func)
print(res1)

{'X':        MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  WindGustSpeed  \
0         13.4     22.9       0.6          4.8       8.4           44.0   
1          7.4     25.1       0.0          4.8       8.4           44.0   
2         12.9     25.7       0.0          4.8       8.4           46.0   
3          9.2     28.0       0.0          4.8       8.4           24.0   
4         17.5     32.3       1.0          4.8       8.4           41.0   
...        ...      ...       ...          ...       ...            ...   
32817     11.6     20.0       0.0          2.0       3.7           33.0   
32818     10.8     20.8       0.0          2.6      10.1           31.0   
32819     11.0     20.2       0.0          4.6       9.3           59.0   
32820      7.3     15.7       0.0          5.0      10.1           63.0   
32825      9.4     22.3       0.0          3.4       9.2           61.0   

       WindSpeed9am  WindSpeed3pm  Humidity9am  Humidity3pm  Pressure9am  \
0              20

In [9]:
res2 = evaluate_model(blender, df_val.drop(columns=[target]), df_val[target], metric_func)
print(res2)

{'X':        MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  WindGustSpeed  \
0         13.4     22.9       0.6          4.8       8.4           44.0   
1          7.4     25.1       0.0          4.8       8.4           44.0   
2         12.9     25.7       0.0          4.8       8.4           46.0   
3          9.2     28.0       0.0          4.8       8.4           24.0   
4         17.5     32.3       1.0          4.8       8.4           41.0   
...        ...      ...       ...          ...       ...            ...   
32817     11.6     20.0       0.0          2.0       3.7           33.0   
32818     10.8     20.8       0.0          2.6      10.1           31.0   
32819     11.0     20.2       0.0          4.6       9.3           59.0   
32820      7.3     15.7       0.0          5.0      10.1           63.0   
32825      9.4     22.3       0.0          3.4       9.2           61.0   

       WindSpeed9am  WindSpeed3pm  Humidity9am  Humidity3pm  Pressure9am  \
0              20